In [1]:
import spacy
import pprint as pp
import os
import sys
from nltk.tokenize import sent_tokenize
import re


In [2]:
path = '/home/folzd/Desktop/'
#path = '/home/folzd/Tale/danish_fairytales_0805_librivox_64kb_mp3/'

In [3]:
def process(filename):
    spurious_periods = map(str,range(1,32))
    abbreviations = ['stk', 'jf', 'pr', 'pkt', 'nr', 'kr', 'mia', 'Hr', 'hr', 'f.eks', 'bl.a']
    one_capital_letter_abbreviations = 'ABCDEFGHIJKLMNOPQRSTVWXYZ'
    spurious_periods.extend(list(one_capital_letter_abbreviations))
    spurious_periods.extend(abbreviations)
    with open(path+filename, 'r') as f: 
        data = f.read().decode("utf-8")
    for spurious_period in spurious_periods:
        data = data.replace(' '+spurious_period+'.',' '+spurious_period.replace('.',''))
    data = data.replace('www.ft.dk','wwwftdk')
    ## pattern: Upercase, then anything that is not in (\-;.!?), then one of them
    paragraphs = re.findall(r'([A-Za-z][^\-;.!?]*[\-;.!?])', data)
    #paragraphs = data.split("\n\n")
    paragraph_sentence_list = []
    for paragraph in paragraphs:
        paragraph = paragraph.replace("\n", " ")
        paragraph = paragraph.replace("--", "")
        paragraph = paragraph.replace(u'\u00A7', "")
        paragraph = paragraph.replace(".", "")
        paragraph = paragraph.replace("!", "")
        paragraph = paragraph.replace("?", "")
        paragraph = paragraph.replace(",", "")
        paragraph = re.sub(r'[\xc9]', 'E', paragraph)
        paragraph = re.sub(r'[\xe9]', 'e', paragraph) # som i "idéen"
        #paragraph = re.sub(r'[^a-zA-Z0-9_*.,?!äöèÄÖÈÉçëË]', ' ', paragraph)
        paragraph = re.sub(r'[^a-zA-Z0-9_*.,?!åäøöèØÆÅÄÖÈÉçëË\xe5\xc5\xc6\xe6\xf8\xd8]', ' ', paragraph)
        paragraph_sentence_list.append(sent_tokenize(paragraph))
    text = ""
    count = 0
    for paragraph in paragraph_sentence_list:
        count += 1
        text += "\n".join(paragraph)
        text += "\n\n"    
    with open(path+filename[:-4]+"_aeneas_data_"+str(count)+".txt", "w") as fw:
        fw.write(text.encode("utf-8"))

In [4]:
onlyfiles = [f for f in os.listdir(path) if f.endswith('txt')]

In [5]:
onlyfiles

['m\xc3\xb8de108.txt']

In [6]:
for filename in onlyfiles:
    process(filename)